
# Recommender Similarity Algorithm Downselect File

## Import Libraries
> We are using the Suprise library because it has all the tools we need to import our dataset, <br>
calculate the similarities and errors from all the similairity algorithms, fit the algorithms <br>
to our dataset, and then output the similarity matrix.

In [245]:
!pip install surprise

In [246]:
import json
import time
import numpy as np
import pandas as pd
from surprise.model_selection import GridSearchCV
from surprise.model_selection import RandomizedSearchCV
from surprise import accuracy
from collections import defaultdict
from surprise import AlgoBase
from surprise import NormalPredictor
from surprise import PredictionImpossible
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise import dump
from surprise.model_selection import cross_validate

## Import Preprocessed User, Restaurant, and Review Data

In [247]:
start = time.time()
with open('MA_users.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    user_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load user data: %d seconds" %duration_without_dr)


Reading MA_users.json
Time taken to load user data: 2 seconds


In [248]:
start = time.time()
with open('MA_restaurants.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    restaurant_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load restaurant data: %d seconds" %duration_without_dr)

Reading MA_restaurants.json
Time taken to load restaurant data: 0 seconds


In [249]:
start = time.time()
with open('MA_reviews.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    review_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load review data: %d seconds" %duration_without_dr)

Reading MA_reviews.json
Time taken to load review data: 6 seconds


In [250]:
print(str(len(user_data)) + ' user records loaded')
print(str(len(review_data)) + ' review records loaded')
print(str(len(restaurant_data)) + ' business records loaded')

125521 user records loaded
914710 review records loaded
10550 business records loaded


Create a 2d arary of user IDs and buisness IDs, with ratings(stars) as frequencies

In [251]:
user_id_dict = dict()
for i,u in enumerate(user_data):
    user_id_dict[u['user_id']] = i

business_id_dict = dict()
for i,r in enumerate(restaurant_data):
    business_id_dict[r['business_id']] = i

In [252]:
rating_mat = np.zeros((len(user_data), len(restaurant_data)))
rating_dict = {'user_id': [],
               'business_id': [],
               'rating': []}

for review in review_data:
    user_idx = user_id_dict[review['user_id']]
    business_idx = business_id_dict[review['business_id']]
    rating_mat[user_idx][business_idx] = review['stars']
    
    rating_dict['user_id'].append(review['user_id'])
    rating_dict['business_id'].append(review['business_id'])
    rating_dict['rating'].append(review['stars'])

print(rating_mat.shape)
print('number of non-zero elements(good ratings): ' +str(np.count_nonzero(rating_mat)))

(125521, 10550)
number of non-zero elements(good ratings): 879189


# This code will be removed and put into preprocessing later

In [253]:
df = pd.DataFrame(rating_dict)

# filter to only the users/businesses with over 50 reviews
# necessary in order to build the anti testset
filter_user = df['user_id'].value_counts() >= 15
filter_user = filter_user[filter_user].index.tolist()

filter_business = df['business_id'].value_counts() > 40
filter_business = filter_business[filter_business].index.tolist()

df_new = df[(df['user_id'].isin(filter_user)) & (df['business_id'].isin(filter_business))]

print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_new[['user_id', 'business_id', 'rating']], reader)

The original data frame shape:	(914710, 3)
The new data frame shape:	(475458, 3)


In [254]:
if 'rating_mat' in globals():
    del rating_mat
    del df

In [255]:
sim_options = {
    "name": ["msd","cosine"],
    "min_support": [1,5],
    "user_based": [False,True]
}
param_grid = {"k": [10, 40],
              "min_k:": [1, 5, 10],
              "sim_options": sim_options}

In [256]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(KNNBasic, param_grid, measures = ["rmse", "mae"], cv=5, n_jobs=3,joblib_verbose=10)
#grid_search.fit(data)

In [257]:
#KNNBasic_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = KNNBasic_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

## KNNBasic Results
Best RSME: 1.0797624464 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Best MAE:  0.8460764842 Best Params:{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}

In [258]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(KNNWithMeans, param_grid, measures = ["rmse", "mae"], cv=5, n_jobs=3,joblib_verbose=10)
#grid_search.fit(data)

In [259]:
#KNNMeans_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = KNNMeans_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

## KNNMeans Results
Best RSME: 1.0367942082 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Best MAE:  0.8085535760 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}

## KNNZScore Grid Search

In [260]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(KNNWithZScore, param_grid, measures = ["rmse", "mae"], cv=5, n_jobs=3,joblib_verbose=10)
#grid_search.fit(data)

In [261]:
#KNNZScore_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = KNNZScore_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

## KNNZScore Results
Best RSME: 1.0387852982 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Best MAE:  0.8090497383 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}

In [262]:
from surprise import SVD
from surprise import SVDpp

In [263]:
sim_options = {
    "n_epochs": [5, 10, 20, 30],
    "lr_all": [.0025, .005, .001, .01]}


In [264]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(SVD, sim_options, measures = ["rmse", "mae"], cv=5, n_jobs=9,joblib_verbose=10)
#grid_search.fit(data)

In [265]:
#SVD_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = SVD_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

## SVD Results

Best RSME: 1.0272185025 Best Params:{'n_epochs': 20, 'lr_all': 0.0025} <br>
Best MAE:  0.8116613978 Best Params:{'n_epochs': 30, 'lr_all': 0.0025}

In [266]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(SVDpp, sim_options, measures = ["rmse", "mae"], cv=5, n_jobs=9,joblib_verbose=10)
#grid_search.fit(data)

In [267]:
#SVDpp_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = SVDpp_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

## SVDpp Results
Best RSME: 1.0230810561 Best Params:{'n_epochs': 30, 'lr_all': 0.0025} <br>
Best MAE:  0.8075366007 Best Params:{'n_epochs': 30, 'lr_all': 0.0025}

In [268]:
sim_options = {
    "name": "msd",
    "min_support": 1,
    "user_based": True
}

algo = KNNBasic(k=40, min_k=10, sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=3, verbose=True);

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0818  1.0787  1.0826  1.0750  1.0797  1.0795  0.0027  
MAE (testset)     0.8511  0.8475  0.8495  0.8436  0.8493  0.8482  0.0026  
Fit time          9.04    9.17    9.18    8.75    8.73    8.98    0.20    
Test time         12.48   12.30   12.42   12.12   12.13   12.29   0.15    


In [269]:
sim_options = {
    "name": "cosine",
    "min_support": 1,
    "user_based": False
}
algo = KNNWithMeans(k=40, min_k=10, sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=3, verbose=True);

Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0361  1.0352  1.0378  1.0397  1.0369  1.0372  0.0015  
MAE (testset)     0.8140  0.8136  0.8159  0.8162  0.8149  0.8149  0.0010  
Fit time          7.64    7.75    7.65    7.41    7.33    7.55    0.16    
Test time         6.19    6.22    6.27    6.08    6.02    6.16    0.09    


In [270]:
sim_options = {
    "name": "cosine",
    "min_support": 1,
    "user_based": False
}
algo = KNNWithZScore(k=40, min_k=10, sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=3, verbose=True);

Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0395  1.0371  1.0360  1.0385  1.0370  1.0376  0.0012  
MAE (testset)     0.8165  0.8139  0.8123  0.8148  0.8148  0.8145  0.0014  
Fit time          7.96    7.83    7.80    7.47    7.47    7.71    0.20    
Test time         6.65    6.67    6.69    6.50    6.55    6.61    0.07    


In [271]:

algo = SVD(n_epochs=20, lr_all=0.0025)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=9, verbose=True);

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0279  1.0294  1.0248  1.0296  1.0268  1.0277  0.0018  
MAE (testset)     0.8137  0.8159  0.8117  0.8150  0.8127  0.8138  0.0015  
Fit time          15.74   15.66   15.72   15.51   15.63   15.65   0.08    
Test time         0.60    0.66    0.70    0.65    0.63    0.65    0.03    


In [272]:
algo = SVDpp(n_epochs=30, lr_all=0.0025)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=9, verbose=True);

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0227  1.0200  1.0257  1.0242  1.0209  1.0227  0.0021  
MAE (testset)     0.8079  0.8051  0.8093  0.8092  0.8043  0.8072  0.0021  
Fit time          381.70  378.62  384.34  373.80  373.58  378.41  4.26    
Test time         4.52    4.50    4.46    4.42    4.39    4.46    0.05    


## Algorithm and Parameter Tuning

### Validation methods
Parameter grid search given the below error measurements:
#### Root Mean Square Error (RMSE)
* Measures standard deviation of errors in set of predictions
* Goal: minimize RMSE

#### Mean Absolute Error (MAE)
* Measures average magnitude of errors in set of predictions
* Goal: minimize MAE

# Backup Data from Runs

## KNNBasic
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    7.9s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   32.7s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   52.2s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.5min <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.4min <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  3.3min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.2min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  4.9min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  6.4min <br>
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  7.9min <br>
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  8.8min <br>
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 10.5min <br>
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 12.2min <br>
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 13.7min <br>
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 16.1min <br>
[Parallel(n_jobs=3)]: Done 175 tasks      | elapsed: 17.6min <br>
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 20.1min <br>
[Parallel(n_jobs=3)]: Done 215 tasks      | elapsed: 22.1min <br>
[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 25.5min finished  <br>

Mean RSME: 1.1207483198 Mean MAE: 0.8704986843 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.1134275198 Mean MAE: 0.8676869132 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1579731854 Mean MAE: 0.8879868382 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1483985917 Mean MAE: 0.9015830440 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1317509528 Mean MAE: 0.8901500859 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.1185394953 Mean MAE: 0.8864770112 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1678848949 Mean MAE: 0.8979010320 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1553289835 Mean MAE: 0.9092083362 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1207483198 Mean MAE: 0.8704986843 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.1134275198 Mean MAE: 0.8676869132 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1579731854 Mean MAE: 0.8879868382 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1483985917 Mean MAE: 0.9015830440 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1317509528 Mean MAE: 0.8901500859 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.1185394953 Mean MAE: 0.8864770112 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1678848949 Mean MAE: 0.8979010320 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1553289835 Mean MAE: 0.9092083362 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1207483198 Mean MAE: 0.8704986843 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.1134275198 Mean MAE: 0.8676869132 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1579731854 Mean MAE: 0.8879868382 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1483985917 Mean MAE: 0.9015830440 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1317509528 Mean MAE: 0.8901500859 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.1185394953 Mean MAE: 0.8864770112 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1678848949 Mean MAE: 0.8979010320 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1553289835 Mean MAE: 0.9092083362 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.0988640938 Mean MAE: 0.8584146262 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.0797624464 Mean MAE: 0.8460764842 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1519888017 Mean MAE: 0.8850846700 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1424616886 Mean MAE: 0.8975418082 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1064197209 Mean MAE: 0.8716636523 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.0854190414 Mean MAE: 0.8598299793 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1583798093 Mean MAE: 0.8930909435 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1484190091 Mean MAE: 0.9044068678 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.0988640938 Mean MAE: 0.8584146262 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.0797624464 Mean MAE: 0.8460764842 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1519888017 Mean MAE: 0.8850846700 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1424616886 Mean MAE: 0.8975418082 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1064197209 Mean MAE: 0.8716636523 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.0854190414 Mean MAE: 0.8598299793 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1583798093 Mean MAE: 0.8930909435 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1484190091 Mean MAE: 0.9044068678 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.0988640938 Mean MAE: 0.8584146262 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.0797624464 Mean MAE: 0.8460764842 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1519888017 Mean MAE: 0.8850846700 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1424616886 Mean MAE: 0.8975418082 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RSME: 1.1064197209 Mean MAE: 0.8716636523 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RSME: 1.0854190414 Mean MAE: 0.8598299793 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RSME: 1.1583798093 Mean MAE: 0.8930909435 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RSME: 1.1484190091 Mean MAE: 0.9044068678 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>  <br>

Best RSME: 1.0797624464 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>  <br>

Best MAE:  0.8460764842 Best Params:{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}

## KNNMeans
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    7.7s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   33.4s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   52.8s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.5min <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.5min <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  3.3min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.2min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  4.9min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  6.4min <br>
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  7.8min <br>
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  8.7min <br>
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 10.4min <br>
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 12.0min <br>
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 13.6min <br>
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 15.9min <br>
[Parallel(n_jobs=3)]: Done 175 tasks      | elapsed: 17.4min <br>
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 19.9min <br>
[Parallel(n_jobs=3)]: Done 215 tasks      | elapsed: 21.8min <br>
[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 25.3min finished <br>  <br>

Mean RSME: 1.0655983535 Mean MAE: 0.8247306086 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0810146844 Mean MAE: 0.8394403437 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0969267715 Mean MAE: 0.8441156753 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1072333151 Mean MAE: 0.8583919258 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0588264076 Mean MAE: 0.8252494764 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0756201468 Mean MAE: 0.8414108802 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0939136988 Mean MAE: 0.8440246289 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1061819101 Mean MAE: 0.8593716276 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0655983535 Mean MAE: 0.8247306086 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0810146844 Mean MAE: 0.8394403437 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0969267715 Mean MAE: 0.8441156753 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1072333151 Mean MAE: 0.8583919258 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0588264076 Mean MAE: 0.8252494764 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0756201468 Mean MAE: 0.8414108802 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0939136988 Mean MAE: 0.8440246289 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1061819101 Mean MAE: 0.8593716276 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0655983535 Mean MAE: 0.8247306086 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0810146844 Mean MAE: 0.8394403437 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0969267715 Mean MAE: 0.8441156753 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1072333151 Mean MAE: 0.8583919258 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0588264076 Mean MAE: 0.8252494764 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0756201468 Mean MAE: 0.8414108802 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0939136988 Mean MAE: 0.8440246289 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1061819101 Mean MAE: 0.8593716276 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0418043176 Mean MAE: 0.8085535760 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0505257691 Mean MAE: 0.8166440026 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0863256661 Mean MAE: 0.8361652569 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.0991071862 Mean MAE: 0.8512321919 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0367942082 Mean MAE: 0.8087903210 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0464099210 Mean MAE: 0.8180279348 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0841756143 Mean MAE: 0.8367188777 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.0989079211 Mean MAE: 0.8526456971 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0418043176 Mean MAE: 0.8085535760 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0505257691 Mean MAE: 0.8166440026 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0863256661 Mean MAE: 0.8361652569 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.0991071862 Mean MAE: 0.8512321919 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0367942082 Mean MAE: 0.8087903210 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0464099210 Mean MAE: 0.8180279348 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0841756143 Mean MAE: 0.8367188777 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.0989079211 Mean MAE: 0.8526456971 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0418043176 Mean MAE: 0.8085535760 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0505257691 Mean MAE: 0.8166440026 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0863256661 Mean MAE: 0.8361652569 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.0991071862 Mean MAE: 0.8512321919 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0367942082 Mean MAE: 0.8087903210 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0464099210 Mean MAE: 0.8180279348 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0841756143 Mean MAE: 0.8367188777 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.0989079211 Mean MAE: 0.8526456971 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br> <br>

Best RSME: 1.0367942082 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br> <br>

Best MAE:  0.8085535760 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}

## KNNZScore
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    8.9s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   37.7s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   58.3s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.6min <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.6min <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  3.6min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.5min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  5.3min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  6.9min <br>
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  8.4min <br>
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  9.4min <br>
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 11.2min <br>
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 12.9min <br>
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 14.6min <br>
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 17.1min <br>
[Parallel(n_jobs=3)]: Done 175 tasks      | elapsed: 18.7min <br>
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 21.4min <br>
[Parallel(n_jobs=3)]: Done 215 tasks      | elapsed: 23.5min <br>
[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 27.1min finished <br> <br>

Mean RSME: 1.0683386376 Mean MAE: 0.8253927641 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0859524368 Mean MAE: 0.8355716867 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.1045790112 Mean MAE: 0.8476680557 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1243669356 Mean MAE: 0.8624770596 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0607615892 Mean MAE: 0.8255806361 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0767696238 Mean MAE: 0.8356348845 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.1016096449 Mean MAE: 0.8474963164 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1218763154 Mean MAE: 0.8624511027 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0683386376 Mean MAE: 0.8253927641 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0859524368 Mean MAE: 0.8355716867 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.1045790112 Mean MAE: 0.8476680557 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1243669356 Mean MAE: 0.8624770596 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0607615892 Mean MAE: 0.8255806361 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0767696238 Mean MAE: 0.8356348845 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.1016096449 Mean MAE: 0.8474963164 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1218763154 Mean MAE: 0.8624511027 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0683386376 Mean MAE: 0.8253927641 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0859524368 Mean MAE: 0.8355716867 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.1045790112 Mean MAE: 0.8476680557 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1243669356 Mean MAE: 0.8624770596 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0607615892 Mean MAE: 0.8255806361 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0767696238 Mean MAE: 0.8356348845 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.1016096449 Mean MAE: 0.8474963164 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1218763154 Mean MAE: 0.8624511027 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0443347945 Mean MAE: 0.8090751138 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0552517612 Mean MAE: 0.8141773828 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0940389812 Mean MAE: 0.8396742721 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1150317967 Mean MAE: 0.8549600818 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0387852982 Mean MAE: 0.8090497383 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0479187969 Mean MAE: 0.8144749330 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0914201809 Mean MAE: 0.8397484110 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1133504557 Mean MAE: 0.8552410299 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0443347945 Mean MAE: 0.8090751138 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0552517612 Mean MAE: 0.8141773828 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0940389812 Mean MAE: 0.8396742721 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1150317967 Mean MAE: 0.8549600818 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0387852982 Mean MAE: 0.8090497383 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0479187969 Mean MAE: 0.8144749330 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0914201809 Mean MAE: 0.8397484110 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1133504557 Mean MAE: 0.8552410299 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0443347945 Mean MAE: 0.8090751138 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0552517612 Mean MAE: 0.8141773828 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0940389812 Mean MAE: 0.8396742721 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1150317967 Mean MAE: 0.8549600818 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RSME: 1.0387852982 Mean MAE: 0.8090497383 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RSME: 1.0479187969 Mean MAE: 0.8144749330 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RSME: 1.0914201809 Mean MAE: 0.8397484110 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RSME: 1.1133504557 Mean MAE: 0.8552410299 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br> <br>

Best RSME: 1.0387852982 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>

Best MAE:  0.8090497383 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}

## SVD
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    6.0s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   15.5s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   21.5s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:   35.3s <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   53.7s <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  1.3min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.1min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  3.0min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  4.4min <br>
[Parallel(n_jobs=3)]: Done  80 out of  80 | elapsed:  6.4min finished <br>

Mean RSME: 1.0553292065 Mean MAE: 0.8416993990 Params{'n_epochs': 5, 'lr_all': 0.0025} <br>
Mean RSME: 1.0370958851 Mean MAE: 0.8244407821 Params{'n_epochs': 5, 'lr_all': 0.005} <br>
Mean RSME: 1.0850222425 Mean MAE: 0.8707043753 Params{'n_epochs': 5, 'lr_all': 0.001} <br>
Mean RSME: 1.0277610396 Mean MAE: 0.8138482030 Params{'n_epochs': 5, 'lr_all': 0.01} <br>
Mean RSME: 1.0374359618 Mean MAE: 0.8247214879 Params{'n_epochs': 10, 'lr_all': 0.0025} <br>
Mean RSME: 1.0273846146 Mean MAE: 0.8137289083 Params{'n_epochs': 10, 'lr_all': 0.005} <br>
Mean RSME: 1.0621503302 Mean MAE: 0.8481700594 Params{'n_epochs': 10, 'lr_all': 0.001} <br>
Mean RSME: 1.0319438457 Mean MAE: 0.8132544921 Params{'n_epochs': 10, 'lr_all': 0.01} <br>
Mean RSME: 1.0272185025 Mean MAE: 0.8134268693 Params{'n_epochs': 20, 'lr_all': 0.0025} <br>
Mean RSME: 1.0321072951 Mean MAE: 0.8134278075 Params{'n_epochs': 20, 'lr_all': 0.005} <br>
Mean RSME: 1.0422773009 Mean MAE: 0.8292470705 Params{'n_epochs': 20, 'lr_all': 0.001} <br>
Mean RSME: 1.0711510227 Mean MAE: 0.8384373243 Params{'n_epochs': 20, 'lr_all': 0.01} <br>
Mean RSME: 1.0273488208 Mean MAE: 0.8116613978 Params{'n_epochs': 30, 'lr_all': 0.0025} <br>
Mean RSME: 1.0505430556 Mean MAE: 0.8246282391 Params{'n_epochs': 30, 'lr_all': 0.005} <br>
Mean RSME: 1.0335123672 Mean MAE: 0.8208200307 Params{'n_epochs': 30, 'lr_all': 0.001} <br>
Mean RSME: 1.0928715037 Mean MAE: 0.8520616481 Params{'n_epochs': 30, 'lr_all': 0.01} <br>

Best RSME: 1.0272185025 Best Params:{'n_epochs': 20, 'lr_all': 0.0025} <br>
Best MAE:  0.8116613978 Best Params:{'n_epochs': 30, 'lr_all': 0.0025}

## SVDpp
[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers. <br>
[Parallel(n_jobs=9)]: Done   7 tasks      | elapsed:  1.5min <br>
[Parallel(n_jobs=9)]: Done  14 tasks      | elapsed:  3.0min <br>
[Parallel(n_jobs=9)]: Done  23 tasks      | elapsed:  5.8min <br>
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  8.5min <br>
[Parallel(n_jobs=9)]: Done  43 tasks      | elapsed: 13.8min <br>
[Parallel(n_jobs=9)]: Done  54 tasks      | elapsed: 19.2min <br>
[Parallel(n_jobs=9)]: Done  72 out of  80 | elapsed: 35.5min remaining:  3.9min <br>
[Parallel(n_jobs=9)]: Done  80 out of  80 | elapsed: 42.0min finished <br>

Mean RSME: 1.0527578934 Mean MAE: 0.8395470539 Params{'n_epochs': 5, 'lr_all': 0.0025} <br>
Mean RSME: 1.0350319500 Mean MAE: 0.8223487040 Params{'n_epochs': 5, 'lr_all': 0.005} <br>
Mean RSME: 1.0817778398 Mean MAE: 0.8688821281 Params{'n_epochs': 5, 'lr_all': 0.001} <br>
Mean RSME: 1.0251171408 Mean MAE: 0.8113078296 Params{'n_epochs': 5, 'lr_all': 0.01} <br>
Mean RSME: 1.0349418847 Mean MAE: 0.8222710249 Params{'n_epochs': 10, 'lr_all': 0.0025} <br>
Mean RSME: 1.0246375321 Mean MAE: 0.8110269266 Params{'n_epochs': 10, 'lr_all': 0.005} <br>
Mean RSME: 1.0595689547 Mean MAE: 0.8462357504 Params{'n_epochs': 10, 'lr_all': 0.001} <br>
Mean RSME: 1.0258545384 Mean MAE: 0.8081739308 Params{'n_epochs': 10, 'lr_all': 0.01} <br>
Mean RSME: 1.0245282081 Mean MAE: 0.8109067178 Params{'n_epochs': 20, 'lr_all': 0.0025} <br>
Mean RSME: 1.0252212612 Mean MAE: 0.8076680217 Params{'n_epochs': 20, 'lr_all': 0.005} <br>
Mean RSME: 1.0400221067 Mean MAE: 0.8271907348 Params{'n_epochs': 20, 'lr_all': 0.001} <br>
Mean RSME: 1.0643620756 Mean MAE: 0.8319213222 Params{'n_epochs': 20, 'lr_all': 0.01} <br>
Mean RSME: 1.0230810561 Mean MAE: 0.8075366007 Params{'n_epochs': 30, 'lr_all': 0.0025} <br>
Mean RSME: 1.0405533039 Mean MAE: 0.8166455465 Params{'n_epochs': 30, 'lr_all': 0.005} <br>
Mean RSME: 1.0314976872 Mean MAE: 0.8187010611 Params{'n_epochs': 30, 'lr_all': 0.001} <br>
Mean RSME: 1.1138983438 Mean MAE: 0.8647625829 Params{'n_epochs': 30, 'lr_all': 0.01} <br>

Best RSME: 1.0230810561 Best Params:{'n_epochs': 30, 'lr_all': 0.0025} <br>
Best MAE:  0.8075366007 Best Params:{'n_epochs': 30, 'lr_all': 0.0025} <br>